In [ ]:
if os.path.exists(dataset_path):
        if os.path.isdir(dataset_path):
            files = glob.glob(os.path.join(dataset_path, "*_clean.jsonl"))
        else:
            files = [dataset_path]

        print(f"Reading files {files}")

        dataset = load_dataset("json", data_files=files, split="train")

    else:
        dataset = load_dataset(dataset_path, split="train", revision=config["revision"] if "revision" in config else None)

这段代码的主要作用是灵活地根据提供的`dataset_path`加载数据集，用于后续的模型训练或评估。具体功能分解如下：

1. **路径检查**：首先，检查`dataset_path`指向的路径是否存在。如果存在：

   - **目录情况**：如果`dataset_path`是一个目录，它会查找该目录下所有以`"_clean.jsonl"`结尾的文件。这意味着预期的数据是以JSONL（JSON Lines）格式存储的，每个文件可能代表一部分数据或某种特定的划分。通过`glob.glob`函数收集这些文件路径。
   
   - **文件情况**：如果`dataset_path`直接指向一个文件，就简单地把这个文件路径放入一个列表中，后续按单个文件处理。

2. **打印信息**：打印即将被读取的文件列表，便于用户了解哪些数据将被加载。

3. **数据加载**：使用`load_dataset`函数根据前面找到的文件或单个文件路径加载数据。如果是从多个JSONL文件加载，指定格式为"json"，并使用`data_files`参数指定所有文件路径，数据分割为"train"。如果最初提供的`dataset_path`是一个直接可由`load_dataset`识别的数据集名称（如Hugging Face数据集仓库中的数据集），则直接使用该路径加载数据集，同时如果有配置中的"revision"字段，则指定数据集的修订版本。

4. **配置兼容性**：在直接从数据集名称加载时，如果配置`config`中包含"revision"键，则使用该修订版本加载数据集，增强了代码的灵活性和对不同数据集版本的支持。

综上所述，这段代码设计得既能够处理本地目录下的多个JSONL格式文件，也能直接加载Hugging Face数据集库中的数据集，适应了不同的数据来源场景，同时也考虑到了数据集的不同版本需求，为后续的模型训练提供了坚实的数据基础。

In [ ]:
 dataset = dataset.train_test_split(test_size=.05, seed=config["seed"])

    train_dataset, val_dataset = dataset["train"], dataset["test"]

这段代码的作用是对已加载的`dataset`进行训练集和验证集的拆分，以便于后续的模型训练和验证。具体步骤如下：

1. **数据集分割**：使用`train_test_split`方法根据给定的比例（在这个例子中是5%作为测试集，即`test_size=.05`）将数据集分为训练集和测试集。这意味着95%的数据将被用作训练数据，剩余的5%用于验证模型性能。这种方法简单直接，适用于初步的数据划分。

2. **设置随机种子**：通过`seed=config["seed"]`参数，确保数据集的分割过程是确定性的。这意味着每次运行代码时，只要`config["seed"]`的值保持不变，相同的数据样本会被分配到训练集或验证集，这对于实验的可复现性至关重要。随机种子确保了数据划分的一致性，使得不同实验之间的结果可以直接比较。

3. **分配数据集**：分割后，代码通过字典访问的方式，将分割出来的数据集分别赋值给`train_dataset`和`val_dataset`。这样，后续的训练循环就可以直接使用这两个数据集，其中`train_dataset`用于模型训练，而`val_dataset`用于在训练过程中评估模型性能，调整超参数，以及监控过拟合等。

总结起来，这段代码实现了数据集的标准化划分，为机器学习或深度学习任务奠定了基础，确保了实验的可复现性和训练/验证流程的清晰组织。

In [ ]:
    if config["streaming"] is False:
        kwargs = {"num_proc": config["num_proc"]}
    else:
        kwargs = {}

这段代码根据配置文件中的`streaming`选项来决定是否向`load_dataset`函数传递额外的参数。其作用在于控制数据集加载过程中的并行处理行为，具体解释如下：

- **非流式处理（Streaming=False）**:
  当`config["streaming"]`设置为`False`时，表示不采用流式方式加载数据。在这种情况下，代码会构建一个包含`num_proc`参数的字典`kwargs`。`num_proc`代表用于数据预处理的进程数。这意味着在加载数据集时，将会并行处理数据，利用多个CPU核心来加速数据的读取和预处理过程，这对于大型数据集尤其有利，可以显著减少数据准备时间。

- **流式处理（Streaming=True）**:
  如果`config["streaming"]`设置为`True`，则表示希望以流式方式加载数据。在流式处理模式下，数据不是一次性全部加载到内存中，而是按需分块加载，这对于内存有限但数据量极大的场景非常有用，可以减少内存占用。这时，`kwargs`被初始化为空字典，因为流式加载通常不支持或不需要指定像`num_proc`这样的并行处理参数。流式加载更侧重于逐条或逐批处理数据，而非并行预处理整个数据集。

综上所述，这段代码根据配置灵活地调整数据加载策略，平衡了数据处理速度、内存占用和处理大规模数据的能力，体现了对不同场景需求的适应性。

In [ ]:

    cols_to_keep = ["input_ids", "labels", "attention_mask"]
    # tokenize inputs and return labels and attention mask
    train_dataset = train_dataset.map(
        lambda ele: tokenize_inputs(config, tokenizer, ele),
        batched=True,
        **kwargs
    )

这段代码的作用是对`train_dataset`中的每个样本应用一个名为`tokenize_inputs`的自定义函数，以便将原始文本数据转换为模型所需的输入格式，并同时生成对应的标签和注意力掩码。以下是具体步骤和目的：

1. **定义要保留的列**：首先，定义了一个列表`cols_to_keep`，它指定了在处理后希望保留的数据列名，这里包括`input_ids`（经过tokenizer编码后的输入序列ID）、`labels`（用于训练时的监督信号，通常与输入部分对齐）和`attention_mask`（标识输入序列中哪些位置是实际的tokens，哪些是填充的padding）。

2. **映射函数应用**：使用`map`方法遍历`train_dataset`中的每个元素（样本）。`map`函数接收一个处理单个样本的函数`tokenize_inputs`，并且通过`batched=True`参数表明这个函数将接收一个批次的样本而不是单个样本，这通常能提高处理效率。这里的`**kwargs`传递了之前根据是否流式处理构建的参数字典，可能包含并行处理的配置。

3. **tokenize_inputs函数**：虽然具体的`tokenize_inputs`函数实现没有给出，但可以根据上下文推测，它的作用是接收配置、tokenizer和数据样本，然后：
   - 对样本中的文本进行分词（tokenization），生成`input_ids`。
   - 依据需求生成`labels`，这可能涉及到对输入文本的某种处理，比如为语言模型任务复制输入作为目标（对于某些任务可能需要不同的处理逻辑）。
   - 计算`attention_mask`，确保模型知道哪些token是实际的输入，哪些是因填充而加入的。

综上，此代码片段的核心目的是将原始文本数据转换为模型训练所需的格式，同时通过批量处理和潜在的并行计算来提高数据预处理的效率。

In [ ]:
remove_cols = [col for col in train_dataset.column_names if col not in cols_to_keep]
    train_dataset = train_dataset.remove_columns(remove_cols)

    val_dataset = val_dataset.map(
        lambda ele: tokenize_inputs(config, tokenizer, ele),
        batched=True,
        **kwargs
    )
    remove_cols = [col for col in val_dataset.column_names if col not in cols_to_keep]
    val_dataset = val_dataset.remove_columns(remove_cols)

    train_dataset = train_dataset.with_format("torch")
    val_dataset = val_dataset.with_format("torch")


这段代码的作用是对`train_dataset`中的每个样本应用一个名为`tokenize_inputs`的自定义函数，以便将原始文本数据转换为模型所需的输入格式，并同时生成对应的标签和注意力掩码。以下是具体步骤和目的：

1. **定义要保留的列**：首先，定义了一个列表`cols_to_keep`，它指定了在处理后希望保留的数据列名，这里包括`input_ids`（经过tokenizer编码后的输入序列ID）、`labels`（用于训练时的监督信号，通常与输入部分对齐）和`attention_mask`（标识输入序列中哪些位置是实际的tokens，哪些是填充的padding）。

2. **映射函数应用**：使用`map`方法遍历`train_dataset`中的每个元素（样本）。`map`函数接收一个处理单个样本的函数`tokenize_inputs`，并且通过`batched=True`参数表明这个函数将接收一个批次的样本而不是单个样本，这通常能提高处理效率。这里的`**kwargs`传递了之前根据是否流式处理构建的参数字典，可能包含并行处理的配置。

3. **tokenize_inputs函数**：虽然具体的`tokenize_inputs`函数实现没有给出，但可以根据上下文推测，它的作用是接收配置、tokenizer和数据样本，然后：
   - 对样本中的文本进行分词（tokenization），生成`input_ids`。
   - 依据需求生成`labels`，这可能涉及到对输入文本的某种处理，比如为语言模型任务复制输入作为目标（对于某些任务可能需要不同的处理逻辑）。
   - 计算`attention_mask`，确保模型知道哪些token是实际的输入，哪些是因填充而加入的。

综上，此代码片段的核心目的是将原始文本数据转换为模型训练所需的格式，同时通过批量处理和潜在的并行计算来提高数据预处理的效率。

In [ ]:
    dataset = dataset.train_test_split(test_size=.05, seed=config["seed"])

    train_dataset, val_dataset = dataset["train"], dataset["test"]

这段代码的作用是从原始的`dataset`中按比例分割出训练集和验证集，具体步骤和目的是：

1. **数据分割**：使用`train_test_split`方法将数据集分割成训练集和验证集。参数`test_size=.05`表示验证集占整个数据集的5%，而剩下的95%将用作训练集。这样的分割比例很常见，尤其是在机器学习和深度学习项目中，验证集用于评估模型在未见数据上的表现，从而帮助调整模型参数和避免过拟合。

2. **设置随机种子**：通过`seed=config["seed"]`确保数据分割的过程是可复现的。提供一个固定的随机种子值意味着每次运行代码时，数据集的分割结果都是一样的。这对于实验的可复现性和结果的一致性至关重要，尤其是在调试模型或对比不同超参数设置时。

3. **分配数据集**：分割完成后，通过键`"train"`和`"test"`从返回的字典中提取出训练集和验证集，分别赋值给`train_dataset`和`val_dataset`。这两个变量随后将用于模型的训练和验证过程中，作为数据输入的来源。

总结来说，这段代码完成了数据集的准备阶段的一个关键步骤——分割数据，确保模型训练和验证流程的正确启动，并且通过设置随机种子保障了实验的可复现性。